# INNER JOIN and LEFT JOIN  

The most defining feature of SQL is arguably the join, as this is what relational databases are really designed to do. While there are several types of join, none are so commonly used as the `INNER JOIN` and `LEFT JOIN`. We will cover these two operators in this section. 

## Setup

First get set up. Download the SQLite database file `company_operations.db` and connect to it. Also bring in `pandas` to display our SQL query results as a `DataFrame`. 

In [1]:
import sqlite3
import pandas as pd
import urllib.request

# download SQLite database and connect to it 
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

## Primary and Foreign Keys

Let's take a look at two tables: `CUSTOMER` and `CUSTOMER_ORDER`. What do they have in common? 

In [2]:
sql = """
SELECT * FROM CUSTOMER
"""

pd.read_sql(sql, conn)


,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,CATEGORY
0,1,Alpha Medical,18745 Train Dr,Dallas,TX,75021,INDUSTRIAL
1,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,GOVERNMENT
2,3,Sports Unlimited,1605 Station Dr,Alexandrai,LA,71301,COMMERCIAL
3,4,Riley Sporting Goods,9854 Firefly Blvd,Austin,TX,78701,COMMERCIAL
4,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,INDUSTRIAL
5,6,Prairie Sports Center,689 Stadium Way,Tulsa,OK,74101,COMMERCIAL
6,7,Facility 95,2396 Runway Dr,Oklahoma City,OK,73101,GOVERNMENT
7,8,Allen Stadium,573 HIllcrest Rd,Allen,TX,75002,COMMERCIAL
8,9,Dent Research,392 45th St,Waco,TX,76700,INDUSTRIAL
9,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,COMMERCIAL


In [3]:
sql = """
SELECT * FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,ORDER_DATE,PRODUCT_ID,QUANTITY,RUSH_SHIP
0,1,9,2021-01-01,7,20,0
1,2,5,2021-01-01,15,110,0
2,3,3,2021-01-01,4,120,0
3,4,6,2021-01-01,7,200,0
4,5,2,2021-01-01,3,60,0
...,...,...,...,...,...,...
1185,1994,9,2021-03-31,4,70,0
1186,1995,5,2021-03-31,8,140,0
1187,1996,10,2021-03-31,7,80,0
1188,1997,9,2021-03-31,6,20,0


The two tables have a `CUSTOMER_ID` column, and you can probably infer what it does for each table. The `CUSTOMER` table has a unique `CUSTOMER_ID` assigned to each customer record. But in the `CUSTOMER_ORDER` table it is used to assign an order to a given `CUSTOMER`, using that `CUSTOMER_ID`. 

This makes the `CUSTOMER` table the **parent table** with the `CUSTOMER_ID` being the **primary key**. The `CUSTOMER_ORDER` table is the **child table** with the `CUSTOMER_ID` being the **foreign key**. You can think of it as the parent table *supplies* data to the child table, via the primary key to the foreign key. 

The primary key cannot have duplicate values, and this makes sense as no two customers should have the same `CUSTOMER_ID`. However, there can be multiple instances of a value in a foreign key column, as a given customer can have multiple orders. This is a classic **one-to-many relationship**. 

These relationships are a designed to be joined on, and are a fundamental use case for `INNER JOIN` and `LEFT JOIN`. 

## INNER JOIN

The `INNER JOIN` is the most common type of join in SQL. It stitches together two or more tables together on one or more fields. In our example, it would be helpful to make our `CUSTOMER_ORDER` records more descriptive, by bringing in `CUSTOMER` information alongside each `CUSTOMER_ORDER` record. An `INNER JOIN` can achieve this as shown below.

In [4]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,ORDER_DATE,PRODUCT_ID,QUANTITY
0,1,9,Dent Research,392 45th St,Waco,TX,76700,2021-01-01,7,20
1,2,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,2021-01-01,15,110
2,3,3,Sports Unlimited,1605 Station Dr,Alexandrai,LA,71301,2021-01-01,4,120
3,4,6,Prairie Sports Center,689 Stadium Way,Tulsa,OK,74101,2021-01-01,7,200
4,5,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-01,3,60
...,...,...,...,...,...,...,...,...,...,...
1185,1994,9,Dent Research,392 45th St,Waco,TX,76700,2021-03-31,4,70
1186,1995,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,2021-03-31,8,140
1187,1996,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,7,80
1188,1997,9,Dent Research,392 45th St,Waco,TX,76700,2021-03-31,6,20


Above we have pulled fields from both `CUSTOMER` and `CUSTOMER_ORDER`, and since the `CUSTOMER_ID` exists in both tables we choose one by using the syntax `CUSTOMER.CUSTOMER_ID` to select the one from the `CUSTOMER` table. 

> For ambiguous fields like `CUSTOMER_ID`, it is a good rule of thumb to prefer the one in the parent table in case you switch your `INNER JOIN` to a left `LEFT JOIN` so it never becomes `NULL`. We will learn about this later. 

The `INNER JOIN` allows us to stitch both tables together and define the commonality using the `ON` keyword, which is where our join condition goes. You can actually specify any condition you want for a `JOIN`, but 99.9% of the time you will likely use a simple equals `=` to line up records between the two tables. 

Another way to think of this is we are copying each `CUSTOMER` record across each respective `CUSTOMER_ORDER` sharing that respective `CUSTOMER_ID`. 

> Occasionally, you might come across colleagues who simply write `JOIN` rather than `INNER JOIN`. This is a shorthand or alias for an `INNER JOIN`, but it is better practice to explicitly express `INNER JOIN` so you make it clear that was the type of join you intended. 

You also should avoid using an old convention of inner joining by selecting tables in a comma-separated way, and using the `WHERE` condition for your join condition as shown below. This is an [inflexible and messy syntax that is less legible](https://stackoverflow.com/questions/1018822/inner-join-on-vs-where-clause), and I encourage avoiding it. 

In [5]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER, CUSTOMER_ORDER
WHERE CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,ORDER_DATE,PRODUCT_ID,QUANTITY
0,1,9,Dent Research,392 45th St,Waco,TX,76700,2021-01-01,7,20
1,2,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,2021-01-01,15,110
2,3,3,Sports Unlimited,1605 Station Dr,Alexandrai,LA,71301,2021-01-01,4,120
3,4,6,Prairie Sports Center,689 Stadium Way,Tulsa,OK,74101,2021-01-01,7,200
4,5,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-01,3,60
...,...,...,...,...,...,...,...,...,...,...
1185,1994,9,Dent Research,392 45th St,Waco,TX,76700,2021-03-31,4,70
1186,1995,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,2021-03-31,8,140
1187,1996,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,7,80
1188,1997,9,Dent Research,392 45th St,Waco,TX,76700,2021-03-31,6,20


## LEFT JOIN

What happens if there are `CUSTOMER` records that do not have any `CUSTOMER_ORDER` records? Do they show up in an `INNER JOIN`? For example, "Alpha Medical" with a `CUSTOMER_ID` of 1 does not have any orders. Does it show up in our `INNER JOIN` query? Let's add a `WHERE` condition to find out. 

In [6]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

WHERE CUSTOMER.CUSTOMER_ID = 1
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,ORDER_DATE,PRODUCT_ID,QUANTITY


Sure enough we get an empty result. But look what happens if we change our `INNER JOIN` to a `LEFT JOIN` (or `LEFT OUTER JOIN` which are both aliases for the same operation). 

In [7]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

WHERE CUSTOMER.CUSTOMER_ID = 1 
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,ORDER_DATE,PRODUCT_ID,QUANTITY
0,None,1,Alpha Medical,18745 Train Dr,Dallas,TX,75021,None,None,None


Note how "Alpha Medical" now shows up with a placeholder record even though it did not have any `CUSTOMER_ORDER` records. All of its `CUSTOMER_ORDER` fields are `NULL` (which Pandas is displaying as `None`) because there were no `CUSTOMER_ORDER` records to join to and populate this information. But the `LEFT JOIN` did append this one placeholder record for "Alpha Medical".

In other words, the `LEFT JOIN` includes all records in the "left" table even if there are no records to join to in the "right" table. By "left" I mean the table literally specified to the left of the `LEFT JOIN` operator. This means the order you declare the tables in your `FROM` matters with a `LEFT JOIN`. 


> There is also a `RIGHT JOIN` or `RIGHT OUTER JOIN` operator, which flips the direction and includes all records in the `RIGHT` table even if there are none to join to in the `LEFT` table. However it is seldom used as what can be done with a `RIGHT JOIN` can also be achieved with a `LEFT JOIN`. There is also a `FULL OUTER JOIN` which includes all records in both directions, but it also is rarely used. As a matter of fact, SQLite does not support the `RIGHT JOIN` or `FULL OUTER JOIN` for this reason. 

As we will see, this can be useful for creating reports later as we likely want to include customers that do not have any orders. Another common use case for `LEFT JOIN` is finding parent records that do not have any children, such as `CUSTOMER` records that do not have any `CUSTOMER_ORDER` records. We can do this by qualifying any `CUSTOMER_ORDER` fields to be null, which normally are not null but consequently become null as a result of the `LEFT JOIN`. 

In [8]:
sql = """
SELECT 
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER_ORDER.CUSTOMER_ID = CUSTOMER.CUSTOMER_ID

WHERE CUSTOMER_ORDER.CUSTOMER_ID IS NULL
"""

pd.read_sql(sql, conn)


,CUSTOMER_ID,CUSTOMER_NAME
0,1,Alpha Medical


## Joining Multiple Tables

What if we wanted to bring `PRODUCT` information to our `CUSTOMER_ORDER` records as well as `CUSTOMER` information? This is possible by executing a second join. Let's take a look at the `PRODUCT` table and note it uses a `PRODUCT_ID`, which also exists in the `CUSTOMER_ORDER` table as a foreign key.

In [9]:
sql = """
SELECT * FROM PRODUCT
"""

pd.read_sql(sql, conn)


,PRODUCT_ID,PRODUCT_NAME,PRODUCT_GROUP,PRICE
0,1,Eagle Kit,ALPHA,120
1,2,Hawkeye Cam,ALPHA,80
2,3,Sparrow Blade,BETA,40
3,4,Raven Klaw,BETA,40
4,5,Kriket Light,GAMMA,25
5,6,Owl NV,ALPHA,100
6,7,Vulture X,BETA,56
7,8,Roadrunner Pro,ALPHA,70
8,9,Falcon Tracker,GAMMA,20
9,10,Emu Handheld,GAMMA,35


Let's bring in the `PRODUCT_NAME` and `PRICE` to show alongside each `CUSTOMER_ORDER`. We can execute a second join on the `PRODUCT_ID` and stitch the `PRODUCT` information table to our existing stitchwork between `CUSTOMER_ORDER` and `CUSTOMER`. 

In [10]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT.PRODUCT_ID,
QUANTITY, 
PRICE

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

INNER JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,ORDER_DATE,PRODUCT_ID,QUANTITY,PRICE
0,1,9,Dent Research,392 45th St,Waco,TX,76700,2021-01-01,7,20,56
1,2,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,2021-01-01,15,110,40
2,3,3,Sports Unlimited,1605 Station Dr,Alexandrai,LA,71301,2021-01-01,4,120,40
3,4,6,Prairie Sports Center,689 Stadium Way,Tulsa,OK,74101,2021-01-01,7,200,56
4,5,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-01,3,60,40
...,...,...,...,...,...,...,...,...,...,...,...
1185,1994,9,Dent Research,392 45th St,Waco,TX,76700,2021-03-31,4,70,40
1186,1995,5,Lite Industrial,462 Roadrunner Blvd,Houston,TX,77254,2021-03-31,8,140,70
1187,1996,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,7,80,56
1188,1997,9,Dent Research,392 45th St,Waco,TX,76700,2021-03-31,6,20,100


You can mix `INNER JOIN` and `LEFT JOIN` in a query, and you have to reason carefully through these scenarios as this becomes use-case specific. But in this scenario, if we wanted to include all `CUSTOMER` records we need to use a `LEFT JOIN` for both joins including for `PRODUCT`, because the null values from the first `LEFT JOIN` will propagate to the next join. The second `LEFT JOIN` will tolerate these null values but the `INNER JOIN` will not and simply omit them. 

In [11]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT.PRODUCT_ID,
QUANTITY, 
PRICE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
"""

pd.read_sql(sql, conn)


,CUSTOMER_ORDER_ID,CUSTOMER_ID,CUSTOMER_NAME,ADDRESS,CITY,STATE,ZIP,ORDER_DATE,PRODUCT_ID,QUANTITY,PRICE
0,NaN,1,Alpha Medical,18745 Train Dr,Dallas,TX,75021,None,NaN,NaN,NaN
1,5.0,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-01,3.0,60.0,40.0
2,16.0,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-01,13.0,60.0,40.0
3,13.0,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-01,15.0,10.0,40.0
4,129.0,2,Oak Cliff Base,2379 Cliff Ave,Abbevile,LA,70510,2021-01-07,13.0,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...
1186,1989.0,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,2.0,90.0,80.0
1187,1975.0,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,2.0,170.0,80.0
1188,1980.0,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,4.0,30.0,40.0
1189,1996.0,10,Gamma Solutions,2752 27th St,Phoenix,AZ,85001,2021-03-31,7.0,80.0,56.0


## Aggregating Joins

If you think of the above queries we ran earlier as "new" tables that were produced by the joins, logically we can apply a `GROUP BY` on them as well as aggregation functions like `SUM()`. If we wanted to find the total revenue by customer, let's add a `PRICE * QUANTITY` expression and name it `REVENUE`. 

In [12]:
sql = """
SELECT
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
PRICE * QUANTITY AS REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
"""

pd.read_sql(sql, conn)


,CUSTOMER_ID,CUSTOMER_NAME,REVENUE
0,1,Alpha Medical,NaN
1,2,Oak Cliff Base,3600.0
2,2,Oak Cliff Base,4800.0
3,2,Oak Cliff Base,8400.0
4,2,Oak Cliff Base,9600.0
...,...,...,...
1186,10,Gamma Solutions,400.0
1187,10,Gamma Solutions,2800.0
1188,10,Gamma Solutions,3600.0
1189,10,Gamma Solutions,5600.0


Then we can `SUM()` that expression and add a `GROUP BY` to roll up the `CUSTOMER` attributes. 

In [13]:
sql = """
SELECT
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
SUM(PRICE * QUANTITY) AS TOTAL_REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY CUSTOMER.CUSTOMER_ID, CUSTOMER_NAME
"""

pd.read_sql(sql, conn)


,CUSTOMER_ID,CUSTOMER_NAME,TOTAL_REVENUE
0,1,Alpha Medical,NaN
1,2,Oak Cliff Base,664660.0
2,3,Sports Unlimited,671070.0
3,4,Riley Sporting Goods,1012460.0
4,5,Lite Industrial,679680.0
5,6,Prairie Sports Center,779870.0
6,7,Facility 95,865410.0
7,8,Allen Stadium,918920.0
8,9,Dent Research,896720.0
9,10,Gamma Solutions,673500.0


Finally, we can `COALESCE()` the `TOTAL_REVENUE` to turn any null values to `0`. And that is it!

In [14]:
sql = """
SELECT
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY CUSTOMER.CUSTOMER_ID, CUSTOMER_NAME
"""

pd.read_sql(sql, conn)


,CUSTOMER_ID,CUSTOMER_NAME,TOTAL_REVENUE
0,1,Alpha Medical,0
1,2,Oak Cliff Base,664660
2,3,Sports Unlimited,671070
3,4,Riley Sporting Goods,1012460
4,5,Lite Industrial,679680
5,6,Prairie Sports Center,779870
6,7,Facility 95,865410
7,8,Allen Stadium,918920
8,9,Dent Research,896720
9,10,Gamma Solutions,673500


And that is it! We have learned the fundamentals of SQL joins. If you get comfortable with this operation, you can call yourself a SQL developer. 

## EXERCISE

Find the total revenue by product by completing the query below, replacing the question marks "?" with the proper SQL.

In [16]:
sql = """
SELECT
PRODUCT.PRODUCT_ID, 
PRODUCT_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM PRODUCT LEFT JOIN CUSTOMER_ORDER
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY PRODUCT.PRODUCT_ID, PRODUCT_NAME
"""

pd.read_sql(sql, conn)


,PRODUCT_ID,PRODUCT_NAME,TOTAL_REVENUE
0,1,Eagle Kit,772800
1,2,Hawkeye Cam,630400
2,3,Sparrow Blade,339200
3,4,Raven Klaw,353600
4,5,Kriket Light,181750
5,6,Owl NV,924000
6,7,Vulture X,425040
7,8,Roadrunner Pro,615300
8,9,Falcon Tracker,183800
9,10,Emu Handheld,272300




### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
SELECT
PRODUCT.PRODUCT_ID, 
PRODUCT_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM PRODUCT LEFT JOIN CUSTOMER_ORDER
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY PRODUCT.PRODUCT_ID, PRODUCT_NAME
"""

pd.read_sql(sql, conn)
